In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats

In [ ]:
psnr_file = "../ANALYZE/EUNAF_EDSRx2_bl_x2_nb16_nf64_st2/Set14RGB/psnr_Set14RGB.npy"
ssim_file = "../ANALYZE/EUNAF_EDSRx2_bl_x2_nb16_nf64_st2/Set14RGB/ssim_Set14RGB.npy"
unc_file = "../ANALYZE/EUNAF_EDSRx2_bl_x2_nb16_nf64_st2/Set14RGB/unc_Set14RGB.npy"